# Script: "RIDER Balancing & Randomization of Conditions" Project: RIDER2b

In [295]:
import os
import numpy as np
from typing import Any, List, Dict, Tuple
import pandas as pd
import random
import matplotlib.pyplot as plt
import copy
from collections import Counter
from collections import defaultdict
from operator import itemgetter
from itertools import groupby
import operator
from pprint import pprint

In [296]:
 #type-setting
Array = np.ndarray

In [294]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [297]:
path = "/Users/born/Documents/Upside_down_task/client/public/stimuli"

# we only want png file
stimuli_name_list = [el for el in os.listdir(path) if os.path.isfile(os.path.join(path, el)) if not(el.endswith("white.png")) and el.endswith('.png')]#one image is called white and cannot be used as a stimulus.
print(type(stimuli_name_list))
n_images = len(stimuli_name_list)
print("There are", n_images, "images in my image folder right now.")
stimuli_name_list

<class 'list'>
There are 110 images in my image folder right now.


['pylon01.png',
 'pipewrench.png',
 'redfox.png',
 'soapdispenser03b.png',
 'stackingring01b.png',
 'desktopcomputer.png',
 'chessknight03b.png',
 'boxingglove01.png',
 'honeybee.png',
 'jeep.png',
 'nyala.png',
 'wheelbarrow01.png',
 'toyhockeystick02a.png',
 'pintofbeer.png',
 'radio03a.png',
 'graduationcap.png',
 'razor04a.png',
 'flipflop01a.png',
 'sailboat.png',
 'rollerblade.png',
 'dragonfly01.png',
 'tabascosauce.png',
 'fryingpan01.png',
 'wateringcan.png',
 'saw02b.png',
 'motoroilbottle04.png',
 'hat04a.png',
 'flamingo.png',
 'bluejay.png',
 'witchhat.png',
 'lynx01.png',
 'mailbox02.png',
 'lock02a.png',
 'zebra.png',
 'armadillo.png',
 'lightbulb01.png',
 'handfan01b.png',
 'thumbtack02b.png',
 'duck01.png',
 'candelabra.png',
 'handbrush02.png',
 'gorilla.png',
 'clownfish.png',
 'paintroller02.png',
 'shoppingcart.png',
 'acousticguitar02.png',
 'snowglobe.png',
 'officechair02.png',
 'windsurfboard.png',
 'outdoorheater.png',
 'megaphone.png',
 'buoy.png',
 'peacock.

In [298]:
# I want the item names to be preceeded by "stimuli/", which makes it easier to use them as excel file later on
string = 'stimuli/'
stimuli_names = [string + x for x in stimuli_name_list]
stimuli_names[1]

'stimuli/pipewrench.png'

In [299]:
def sample_stimuli(stimuli_names: List[str], num_stimuli: int) -> List[str]:
    """Sample <num_stimuli> stimuli from the set of all stimuli without replacement."""
    sampled_stimuli = np.random.choice(a=stimuli_names, size=num_stimuli, replace=False).tolist()
    assert len(set(sampled_stimuli)) == len(sampled_stimuli), '\nStimuli must be unique.\n'
    return sampled_stimuli

In [300]:
def add_orientations_to_stimuli(sampled_stimuli: List[str], orientations: List[str]):
    """Add the orientation directly to the string of the stimuli to enable fixed connection between stimuli and orientations"""
    stimuli_and_orientations = list()
    for stim in sampled_stimuli:
        orientation = np.random.choice(a=possible_orientations, size=1, replace=False)
        sampled_stimuli = [stim + str(orientation)]
        stimuli_and_orientations.append(sampled_stimuli)
    return stimuli_and_orientations

In [301]:
def remove_stimuli(stimuli: List[str], sampled_stimuli: List[str]) -> None:
    """Remove stimuli that were sampled for a single trial from the set of possible exp. stimuli."""
    for stimulus in sampled_stimuli:
        stimuli.pop(stimuli.index(stimulus))

In [302]:
def partition_stimuli_into_experimental_conditions(
    stimuli: List[str],
    num_conditions: int,
    num_trials_per_condition: int
) -> Dict[str, List[str]]:
    """Partition stimuli into the five different conditions."""
    condition_partitioning = {}
    stimuli_copy = copy.deepcopy(stimuli)
    for i in range(1, num_conditions+1):
        # one stimulus is needed per condition
        num_stimuli = num_trials_per_condition
        condition_stimuli = np.random.choice(a=stimuli_copy, size=num_stimuli, replace=False).tolist()
        remove_stimuli(stimuli=stimuli_copy, sampled_stimuli=condition_stimuli)
        condition_partitioning[f'condition{i}'] = condition_stimuli
    print(len(stimuli))
    return condition_partitioning

In [12]:
condition_partitioning = partition_stimuli_into_experimental_conditions(
    stimuli=sampled_stimuli,
# we need to partition the stimuli across all 5 conditions, although trial type 3 test 2 will be "invisible"
    num_conditions= 5,
    num_trials_per_condition=22
)
condition_partitioning

NameError: name 'sampled_stimuli' is not defined

In [303]:
def rotate_stimuli(
    condition_partitioning: Dict[str, List[str]],
    num_rotation_stimuli: int
) -> Dict[str, List[str]]:
    """Rotate stimuli through conditions by <num_rotation_stimuli>."""
    condition1_stimuli = condition_partitioning['condition1']
    condition2_stimuli = condition_partitioning['condition2']
    condition3_stimuli = condition_partitioning['condition3']
    condition4_stimuli = condition_partitioning['condition4']
    condition5_stimuli = condition_partitioning['condition5']
    
    condition_partitioning['condition1'] = condition5_stimuli[:num_rotation_stimuli]
    condition_partitioning['condition2'] = condition1_stimuli[:num_rotation_stimuli]
    condition_partitioning['condition3'] = condition2_stimuli[:num_rotation_stimuli]
    condition_partitioning['condition4'] = condition3_stimuli[:num_rotation_stimuli]
    condition_partitioning['condition5'] = condition4_stimuli[:num_rotation_stimuli]
   
    return condition_partitioning

In [14]:
def rotate_stimuli(
    condition_partitioning: Dict[str, List[str]],
    num_rotation_stimuli: int
) -> Dict[str, List[str]]:
    """Rotate stimuli through conditions by <num_rotation_stimuli>."""
    condition_keys = list(condition_partitioning.keys())
    num_conditions = len(condition_keys)
    
    for i in range(num_conditions - 1, 0, -1):
        source_key = condition_keys[i]
        target_key = condition_keys[i - 1]
        
        source_stimuli = condition_partitioning[source_key]
        target_stimuli = condition_partitioning[target_key]
        
        condition_partitioning[target_key] = source_stimuli[:num_rotation_stimuli]
        condition_partitioning[source_key] = target_stimuli[:num_rotation_stimuli]
    
    condition5_stimuli = condition_partitioning['condition5']
    selected_stimuli = random.sample(condition5_stimuli, 11)
    condition_partitioning['condition5'] = selected_stimuli + selected_stimuli
    condition_partitioning['condition1'] = condition5_stimuli[:num_rotation_stimuli]
    
    return condition_partitioning

In [15]:
condition_partitioning = rotate_stimuli(condition_partitioning, num_rotation_stimuli = 22)

NameError: name 'condition_partitioning' is not defined

In [613]:
condition_partitioning

{'condition1': ['stimuli/handbrush02.png[33.75]',
  'stimuli/fireextinguisher02.png[348.75]',
  'stimuli/wheelbarrow01.png[236.25]',
  'stimuli/parrot01.png[303.75]',
  'stimuli/tabascosauce.png[281.25]',
  'stimuli/antlers.png[146.25]',
  'stimuli/comb02b.png[213.75]',
  'stimuli/flamingo.png[213.75]',
  'stimuli/redfox.png[213.75]',
  'stimuli/chisel01b.png[78.75]',
  'stimuli/chessknight03b.png[146.25]',
  'stimuli/handbrush02.png[33.75]',
  'stimuli/fireextinguisher02.png[348.75]',
  'stimuli/wheelbarrow01.png[236.25]',
  'stimuli/parrot01.png[303.75]',
  'stimuli/tabascosauce.png[281.25]',
  'stimuli/antlers.png[146.25]',
  'stimuli/comb02b.png[213.75]',
  'stimuli/flamingo.png[213.75]',
  'stimuli/redfox.png[213.75]',
  'stimuli/chisel01b.png[78.75]',
  'stimuli/chessknight03b.png[146.25]'],
 'condition2': ['stimuli/pipewrench.png[281.25]',
  'stimuli/handbrush02.png[33.75]',
  'stimuli/fireextinguisher02.png[348.75]',
  'stimuli/antlers.png[146.25]',
  'stimuli/chisel01b.png[78.

In [304]:
def sample_condition(stimuli: List[str], experiment_condition: str) -> Dict[str, str]:
    """Sample a single condition."""
    # initialize an empty dictionary to store all variables necessary for a single trial
    condition = {}
    if(experiment_condition == 'condition1'):
        # initialize a placeholder variable for 'image2' and 'test2'
        placeholder = '_'
        # draw a single stimulus for presenting images and testing retrieval 
        sampled_stimuli = random.choices(stimuli, k=1)
        condition['image1'] = sampled_stimuli[0]
        condition['image2'] = placeholder
        condition['test1']  = sampled_stimuli[0]
        condition['test2']  = placeholder
        condition['baselinePresent'] = '1'
        condition['baselinePresent2'] = '0'
        condition['adjust2Present'] = '0'
        condition['trial_type'] = 'baseline'
    elif(experiment_condition == 'condition2'):
        # draw one stimulus from the set of experimental stimuli without replacement
        sampled_stimuli = random.choices(stimuli, k=1)
        condition['image1'] = sampled_stimuli[0]
        condition['test1']  = sampled_stimuli[0]
        condition['baselinePresent'] = '0'
        condition['baselinePresent2'] = '1'
        condition['adjust2Present'] = '1'
        condition['trial_type'] = 't2'
    elif(experiment_condition == 'condition3'):
        # draw one stimulus from the set of experimental stimuli without replacement
        sampled_stimuli = random.choices(stimuli, k=1)
        condition['image2'] = sampled_stimuli[0]
        condition['test2']  = sampled_stimuli[0]
        condition['trial_type'] = 't2'
    elif(experiment_condition == 'condition4'):
        placeholder = '_'
        # draw one stimulus from the set of experimental stimuli without replacement
        sampled_stimuli = random.choices(stimuli, k=1)
        # randomly permute the order of the two stimuli for presenting images to participants
        condition['image1'] = sampled_stimuli[0]
        # randomly select one stimulus that is shown as image2
        condition['test1'] = sampled_stimuli[0]
        condition['test2'] = placeholder
        condition['baselinePresent'] = '0'
        condition['baselinePresent2'] = '1'
        condition['adjust2Present'] = '0'
        condition['trial_type'] = 't3'
    elif(experiment_condition == 'condition5'):
        sampled_stimuli = random.choices(stimuli, k=1)
        condition['image2'] = sampled_stimuli[0]
        condition['trial_type'] = 't3'
        # note that here is nothing following for condition5 because these stimuli are not shown in the WM experiment
        
    else:
        raise ValueError('\nThere are 5 conditions.\nUse condition1,condition2,condition3,condition4 or condition5 .\n')
    # remove sampled stimuli from the participant's set of stimuli so that they are not drawn again
    remove_stimuli(stimuli=stimuli, sampled_stimuli=sampled_stimuli)
    return condition

In [305]:
def sample_conditions(
    condition_order: List[str], 
    condition_sets: Dict[str, List[str]],
) -> List[str]:
    """Sample stimuli for all trials in a practice or experimental session."""
    sampled_conditions = [sample_condition(condition_sets[condition], condition) for condition in condition_order]
    return sampled_conditions

In [306]:
def participant_randomization(
    sessions: Dict[str, int],
    condition_sets: Dict[str, List[str]],
    conditions: List[str],
    num_conditions: int,
) -> List[str]:
    """Sample practice and experimental conditions for a single participant."""
    participant_conditions = []
    for session, number_conditions in sessions.items():
        # number of conditions per type
        num_conditions_per_type = number_conditions // len(conditions)
        # extend the list of condition types by <num_conditions_per_type>
        condition_order = conditions * num_conditions_per_type
        # randomly permute the order of condition types for a single session
        random.shuffle(condition_order)
        sampled_conditions = sample_conditions(condition_order,condition_sets)
        participant_conditions.extend(sampled_conditions)        
    #participant_conditions = pd.DataFrame(participant_conditions)
    print(num_conditions_per_type)
    print(len(conditions))
    print(number_conditions)
    print(condition_order)
    return participant_conditions

In [307]:
def stimuli_balancing(
    num_participants: int,
    sessions: Dict[str, int],
    condition_partitioning: Dict[str, List[str]],
    conditions: List[str],
) -> List[str]:
    """
    This is the main loop for balancing the experimental conditions. 
    Here, we sample both practice and experimental conditions for all participants.
    """
    num_conditions = len(conditions)
    particpant_randomizations = []
    for i in range(1, num_participants+1):
        if i > 1:
            # rotate stimuli through the three type sets by <num_rotation_stimuli> (currently 22)
            condition_partitioning = rotate_stimuli(condition_partitioning, num_rotation_stimuli = 22)
        # create copies of the three trial type sets so that we can remove stimuli 
        # from the three trial type sets without changing the original sets
        condition_partitioning_copy = copy.deepcopy(condition_partitioning)
        # sample practice and experimental trials for a single participant
        participant_conditions = participant_randomization(
            sessions=sessions,
            condition_sets=condition_partitioning_copy,
            conditions=conditions,
            num_conditions=num_conditions,
        )
        # add start orientations to conditions file
        #participant_trials['startOri1'] = start_orientations[:, 0]
        #participant_trials['startOri2'] = start_orientations[:, 1]
        #participant_trials['testOri1']  = start_orientations[:, 2]
        #participant_trials['testOri2']  = start_orientations[:, 3]
        
        # permute the order of the presented stimuli (i.e., shuffle rows) to counteract potential orientation biases
        #participant_conditions = participant_conditions.sample(frac=1, replace=False).reset_index(drop=True)
        particpant_randomizations.append(participant_conditions)
    return particpant_randomizations

In [308]:
# seed random number generator
rnd_seed = 40 #42
random.seed(rnd_seed)
np.random.seed(rnd_seed)

In [309]:
#### define experimental variables ####
num_stimuli = 110
num_practice_stimuli = 10
num_exp_stimuli = num_stimuli - num_practice_stimuli
num_participants = 5

ttypes = ['t1', 't2', 't3']
conditions = ['condition1','condition2','condition3','condition4', 'condition5']
num_practice_conditions = 0
num_exp_conditions = 110
num_total_conditions = num_practice_conditions + num_exp_conditions
sessions = {'practice': num_practice_conditions, 'experimental': num_exp_conditions}

stimulus_order = ['stim1','stim2','stim3','stim4','stim5']

# compute number of trials per trial type
num_condition_per_type =  22# num_total_trials // len(ttypes)

# define variables for orientations of the stimuli
min_orientation = 11.25
max_orientation = 371.25
step_size =  22.5

possible_orientations = np.arange(min_orientation, max_orientation, step_size)
possible_orientations = possible_orientations.tolist()
possible_orientations = possible_orientations * 7
len(possible_orientations)

112

In [561]:
# sample stimuli without replacement
sampled_stimuli = sample_stimuli(stimuli_names=stimuli_names, num_stimuli=num_stimuli)
sampled_stimuli

['stimuli/globe.png',
 'stimuli/comb02b.png',
 'stimuli/dragonfly01.png',
 'stimuli/sailboat.png',
 'stimuli/fireextinguisher02.png',
 'stimuli/lightbulb01.png',
 'stimuli/lock02a.png',
 'stimuli/bicycle.png',
 'stimuli/rhinoceros02.png',
 'stimuli/fryingpan01.png',
 'stimuli/megaphone.png',
 'stimuli/scrubbingbrush05a.png',
 'stimuli/graduationcap.png',
 'stimuli/corkscrew03a.png',
 'stimuli/buoy.png',
 'stimuli/greywolf.png',
 'stimuli/saw02b.png',
 'stimuli/bassethound.png',
 'stimuli/officechair02.png',
 'stimuli/dromedary.png',
 'stimuli/handfan01b.png',
 'stimuli/outdoorheater.png',
 'stimuli/windsurfboard.png',
 'stimuli/gardengnome01.png',
 'stimuli/shoppingcart.png',
 'stimuli/aloe01.png',
 'stimuli/pylon01.png',
 'stimuli/badmintonracket.png',
 'stimuli/parrot01.png',
 'stimuli/bluejay.png',
 'stimuli/watercooler.png',
 'stimuli/stapler03b.png',
 'stimuli/flipflop01a.png',
 'stimuli/jeep.png',
 'stimuli/snowglobe.png',
 'stimuli/acousticguitar02.png',
 'stimuli/photocopier.pn

In [562]:
# add orientations to stimuli that can be extracted to independant variables later
sampled_stimuli = add_orientations_to_stimuli(sampled_stimuli = sampled_stimuli, orientations = possible_orientations)
# flatten List of Lists Using a List Comprehension
sampled_stimuli = [item for sublist in sampled_stimuli for item in sublist]

In [563]:
sampled_stimuli

['stimuli/globe.png[146.25]',
 'stimuli/comb02b.png[326.25]',
 'stimuli/dragonfly01.png[191.25]',
 'stimuli/sailboat.png[236.25]',
 'stimuli/fireextinguisher02.png[191.25]',
 'stimuli/lightbulb01.png[123.75]',
 'stimuli/lock02a.png[326.25]',
 'stimuli/bicycle.png[236.25]',
 'stimuli/rhinoceros02.png[123.75]',
 'stimuli/fryingpan01.png[11.25]',
 'stimuli/megaphone.png[101.25]',
 'stimuli/scrubbingbrush05a.png[56.25]',
 'stimuli/graduationcap.png[326.25]',
 'stimuli/corkscrew03a.png[33.75]',
 'stimuli/buoy.png[78.75]',
 'stimuli/greywolf.png[33.75]',
 'stimuli/saw02b.png[236.25]',
 'stimuli/bassethound.png[236.25]',
 'stimuli/officechair02.png[123.75]',
 'stimuli/dromedary.png[168.75]',
 'stimuli/handfan01b.png[258.75]',
 'stimuli/outdoorheater.png[33.75]',
 'stimuli/windsurfboard.png[56.25]',
 'stimuli/gardengnome01.png[168.75]',
 'stimuli/shoppingcart.png[11.25]',
 'stimuli/aloe01.png[101.25]',
 'stimuli/pylon01.png[213.75]',
 'stimuli/badmintonracket.png[168.75]',
 'stimuli/parrot01.p

In [564]:
# partition the stimuli into three disjoint sets, one for each trial type
condition_partitioning = partition_stimuli_into_experimental_conditions(
    stimuli=sampled_stimuli,
# we need to partition the stimuli across all 5 conditions, although trial type 3 test 2 will be "invisible"
    num_conditions= 5,
    num_trials_per_condition=22
)
condition_partitioning

110


{'condition1': ['stimuli/stapler03b.png[56.25]',
  'stimuli/corkscrew03a.png[33.75]',
  'stimuli/outdoorheater.png[33.75]',
  'stimuli/africanelephant.png[101.25]',
  'stimuli/toyhockeystick02a.png[168.75]',
  'stimuli/apron.png[56.25]',
  'stimuli/nyala.png[348.75]',
  'stimuli/pintofbeer.png[101.25]',
  'stimuli/stackingring01b.png[326.25]',
  'stimuli/buoy.png[78.75]',
  'stimuli/hat04a.png[348.75]',
  'stimuli/razor04a.png[146.25]',
  'stimuli/needlenosepliers03b.png[11.25]',
  'stimuli/bassethound.png[236.25]',
  'stimuli/blender.png[11.25]',
  'stimuli/toytractor02a.png[168.75]',
  'stimuli/pear01.png[11.25]',
  'stimuli/acorn.png[258.75]',
  'stimuli/radio03a.png[146.25]',
  'stimuli/barbecue02.png[146.25]',
  'stimuli/dromedary.png[168.75]',
  'stimuli/gecko.png[258.75]'],
 'condition2': ['stimuli/greywolf.png[33.75]',
  'stimuli/accordion02.png[213.75]',
  'stimuli/paintroller02.png[236.25]',
  'stimuli/motoroilbottle04.png[191.25]',
  'stimuli/peacock.png[56.25]',
  'stimuli/

In [565]:
# get all participant condition randomizations
participant_randomizations = stimuli_balancing(
    num_participants=5,
    sessions=sessions, 
    condition_partitioning= condition_partitioning,
    conditions=conditions, 
)

22
5
110
['condition3', 'condition1', 'condition5', 'condition2', 'condition4', 'condition4', 'condition2', 'condition3', 'condition3', 'condition2', 'condition3', 'condition5', 'condition5', 'condition3', 'condition2', 'condition3', 'condition4', 'condition1', 'condition5', 'condition4', 'condition4', 'condition1', 'condition4', 'condition4', 'condition1', 'condition1', 'condition2', 'condition5', 'condition5', 'condition2', 'condition2', 'condition4', 'condition4', 'condition2', 'condition2', 'condition2', 'condition4', 'condition2', 'condition2', 'condition5', 'condition1', 'condition2', 'condition3', 'condition5', 'condition2', 'condition1', 'condition5', 'condition3', 'condition3', 'condition3', 'condition2', 'condition1', 'condition5', 'condition1', 'condition2', 'condition3', 'condition3', 'condition3', 'condition5', 'condition3', 'condition1', 'condition4', 'condition5', 'condition4', 'condition1', 'condition4', 'condition3', 'condition4', 'condition3', 'condition2', 'condition

In [566]:
# Sort conditions in trial types
def sort_conditions_in_trial_types(
    num_participants: int,
    particpant_randomizations: List[str],
)-> List[str]:
    """Sort conditions in trial type 1 (baseline) and 3"""
    trial_types = itemgetter('trial_type')
    participant_grouped_conditions = {}
    # outer loop: loop over each randomization
    for i, randomization in enumerate(particpant_randomizations, start=1):
        trial_type_grouping = defaultdict(list)
         # inner loop: loop over every condition in current subject's randomization
        for condition in randomization:
            trial_type_grouping[trial_types(condition)].append(condition)
        participant_grouped_conditions[f'participant_{i:02d}'] = trial_type_grouping
        
    """
    trial_types = itemgetter('trial_type')
    trial_type_grouping = defaultdict(lambda: defaultdict(list))
    for i, randomization in enumerate(particpant_randomizations, start=1):
        for condition in randomization:
            trial_type_grouping[f'participant_{i:02d}'][trial_types(condition)].append(condition)
    """
    return participant_grouped_conditions

In [567]:
sorted_conditions = sort_conditions_in_trial_types(num_participants = 5 ,particpant_randomizations=participant_randomizations)



In [627]:
#participant_dfs = {}

#for participant_key, conditions in sorted_conditions.items():
    #max_length = max(len(arr) for arr in conditions.values())
    #filled_conditions = {key: arr + [None] * (max_length - len(arr)) for key, arr in conditions.items()}
    #participant_df = pd.DataFrame(filled_conditions)
    #participant_dfs[participant_key] = participant_df

#first_participant_df = participant_dfs['participant_01']
#print(first_participant_df)

                                                   t3  \
0   {'image2': 'stimuli/fireextinguisher02.png[348...   
1   {'image2': 'stimuli/chameleon.png[213.75]', 't...   
2   {'image1': 'stimuli/honeybee.png[123.75]', 'te...   
3   {'image1': 'stimuli/nyala.png[213.75]', 'test1...   
4   {'image1': 'stimuli/handfan01b.png[236.25]', '...   
5   {'image2': 'stimuli/lightbulb01.png[258.75]', ...   
6   {'image2': 'stimuli/sailboat.png[191.25]', 'tr...   
7   {'image2': 'stimuli/bluejay.png[56.25]', 'tria...   
8   {'image2': 'stimuli/maraca02a.png[78.75]', 'tr...   
9   {'image1': 'stimuli/pear01.png[191.25]', 'test...   
10  {'image2': 'stimuli/soapdispenser03b.png[281.2...   
11  {'image2': 'stimuli/acorn.png[101.25]', 'trial...   
12  {'image2': 'stimuli/motoroilbottle04.png[78.75...   
13  {'image2': 'stimuli/monument.png[191.25]', 'tr...   
14  {'image2': 'stimuli/handbrush02.png[281.25]', ...   
15  {'image1': 'stimuli/hen.png[191.25]', 'test1':...   
16  {'image2': 'stimuli/desktop

In [681]:
# From here it goes participant per participant
baseline_trials       = sorted_conditions['participant_05']['baseline']
t3_trials             = sorted_conditions['participant_05']['t3']
t2_trials             = sorted_conditions['participant_05']['t2']

# to construct the final trials I need to extract the trial variables that I need from the conditions
test2_stimuli      = [stimulus['test2'] for stimulus in t2_trials if 'test2' in stimulus]
test1_stimuli      = [stimulus['test1'] for stimulus in t2_trials if 'test1' in stimulus]

image2_stimuli     = [stimulus['image2'] for stimulus in t3_trials if 'image2' in stimulus]
test1_stimuli_t3   = [stimulus['test1'] for stimulus in t3_trials if 'test1' in stimulus]


def define_t2_trials(stimuli_t1: List[str],stimuli_t2: List[str]) -> List[str]:
    """Sample a single t2 trial"""
    # initialize an empty dictionary to store all variables necessary for a single trial
    trial2 = {}
    for stimulus in range(len(stimuli_t1)):
        # draw a single stimulus for presenting images and testing retrieval
        sampled_stim_t1 = random.choices(stimuli_t1, k=1)
        sampled_stim_t2 = random.choices(stimuli_t2, k=1)
        trial2['image1'] = sampled_stim_t1[0]
        trial2['image2'] = sampled_stim_t2[0]
        trial2['test1'] = sampled_stim_t1[0]
        trial2['test2'] = sampled_stim_t2[0]
        trial2['baselinePresent'] = '0'
        trial2['baselinePresent2'] = '1'
        trial2['adjust2Present'] = '1'
        trial2['trial_type'] = 't2'
        trial2.update(trial2)
    remove_stimuli(stimuli=stimuli_t1, sampled_stimuli=sampled_stim_t1)
    remove_stimuli(stimuli=stimuli_t2, sampled_stimuli=sampled_stim_t2)
    return trial2

def sample_t2_trials(
    stimuli_test1: List[str],
    stimuli_test2: List[str],
    stimulus_order: List[str],
) -> List[str]:
    """Sample stimuli for all t2 trials in a practice or experimental session."""
    sampled_t2_trials = [define_t2_trials(stimuli_t1 = test1_stimuli, stimuli_t2 = test2_stimuli) for stim in stimulus_order]
    return sampled_t2_trials

def define_t3_trials(stimuli_t1: List[str],stimuli_i2: List[str]) -> List[str]:
    """Sample a single t3 trial"""
    # initialize an empty dictionary to store all variables necessary for a single trial
    trial3 = {}
    placeholder = '_'
    for stimulus in range(len(stimuli_t1)):
        # draw a single stimulus for presenting images and testing retrieval
        sampled_stim_t1 = random.choices(stimuli_t1, k=1)
        sampled_stim_i2 = random.choices(stimuli_i2, k=1)
        trial3['image1'] = sampled_stim_t1[0]
        trial3['image2'] = sampled_stim_i2[0]
        trial3['test1'] = sampled_stim_t1[0]
        trial3['test2'] = placeholder
        trial3['baselinePresent'] = '0'
        trial3['baselinePresent2'] = '1'
        trial3['adjust2Present'] = '0'
        trial3['trial_type'] = 't3'
        trial3.update(trial3)
    remove_stimuli(stimuli=stimuli_t1, sampled_stimuli=sampled_stim_t1)
    remove_stimuli(stimuli=stimuli_i2, sampled_stimuli=sampled_stim_i2)
    return trial3

def sample_t3_trials(
    stimuli_test1: List[str],
    stimuli_image2: List[str],
    stimulus_order: List[str],
) -> List[str]:
    """Sample stimuli for all t3 trials in a practice or experimental session."""
    sampled_t3_trials = [define_t3_trials(stimuli_t1 = test1_stimuli_t3, stimuli_i2 = image2_stimuli) for stim in stimulus_order]
    return sampled_t3_trials


stims = ['stim1','stim2']
stimulus_order = ['stim1','stim2','stim3','stim4','stim5']*4 + stims

t2_trials = sample_t2_trials(stimuli_test1=test1_stimuli,stimuli_test2=test2_stimuli,stimulus_order =stimulus_order)
t3_trials = sample_t3_trials(stimuli_test1=test1_stimuli_t3,stimuli_image2=image2_stimuli,stimulus_order =stimulus_order)
trials = t2_trials + t3_trials + baseline_trials
random.shuffle(trials)
len(stimulus_order)

# convert the list of trials into a pandas dataframe
participant_trials = pd.DataFrame(trials)


participant_trials["swap"] = np.random.choice(["Yes", "No"], len(participant_trials), p=[0.5, 0.5])
participant_trials

#participant_trials[['image1','image2']] = pd.np.select(participant_trials['swap'] == 0, participant_trials[['image2','image1']].values, participant_trials[['image1','image2']].values)
#participant_trials[['image1','image2']] = participant_trials[['image2','image1']].where((participant_trials['swap'] == 'Yes') & (participant_trials['trial_type'] == 't2'), participant_trials[['image1','image2']].values)
#participant_trials[['image1','image2']] = participant_trials[['image2','image1']].where((participant_trials['swap'] == 'Yes') & (participant_trials['trial_type'] == 't3'), participant_trials[['image1','image2']].values)


# sample 6 practice trials 
size = 2        # sample size
replace = False  # with replacement
fn = lambda obj: obj.loc[np.random.choice(obj.index, size, replace),:]
training = participant_trials.groupby('trial_type', as_index=False).apply(fn)
training = training.sample(frac = 1)

# create final WM conditions file with 6 trainings trials in the beginning (two of each trial type)
trials_training = participant_trials['image1'].isin(training['image1'])

participant_trials.drop(participant_trials[trials_training].index, inplace = True)


# Select the rows where trial_type is 't3'
t3_rows = participant_trials[participant_trials['trial_type'] == 't3']

# Randomly select 10 image names from image2 column
selected_images = random.sample(list(t3_rows['image2']), k=10)

# Overwrite the image names in the other 10 rows of 't3' with the selected images
participant_trials.loc[participant_trials['trial_type'] == 't3', 'image2'] = selected_images * 2



trials_exp = participant_trials
print(len(trials_exp))
print(len(participant_trials))
participant_trials = pd.concat([training,participant_trials.loc[:]]).reset_index(drop=True)
print(len(participant_trials))




participant_trials[['image1','image2']] = participant_trials[['image2','image1']].where((participant_trials['swap'] == 'Yes') & (participant_trials['trial_type'] == 't2'), participant_trials[['image1','image2']].values)
participant_trials[['image1','image2']] = participant_trials[['image2','image1']].where((participant_trials['swap'] == 'Yes') & (participant_trials['trial_type'] == 't3'), participant_trials[['image1','image2']].values)




# generate ltm file
trials_exp = pd.DataFrame(trials_exp)
conditions_ltm_image1 = trials_exp[['trial_type','image1',]]
conditions_ltm_image2 = trials_exp[['trial_type','image2',]]
conditions_ltm_image2 = conditions_ltm_image2[conditions_ltm_image2['image2'] != '_']
conditions_ltm_image2 = conditions_ltm_image2.rename(columns={'image2': 'image1'})
conditions_ltm_ = pd.concat([conditions_ltm_image1,conditions_ltm_image2.loc[:]]).reset_index(drop=True)


conditions_ltm_['startOri_ltm'] = conditions_ltm_['image1'].str.split('[',1).str[1].str.strip()
conditions_ltm_['startOri_ltm'] = conditions_ltm_['startOri_ltm'].str.split(']',0).str[0].str.strip()
conditions_ltm_['ltm_image'] = conditions_ltm_['image1'].str.split('[',0).str[0].str.strip()
conditions_ltm_['startOri_ltm'] = conditions_ltm_['startOri_ltm'].astype(float)
conditions_ltm_


# finish creating WM conditions file
# split orientation and object identity also in working memory experiment
participant_trials['startOri1'] = participant_trials['image1'].str.split('[',1).str[1].str.strip()
participant_trials['startOri1'] = participant_trials['startOri1'].str.split(']',0).str[0].str.strip()
participant_trials['startOri2'] = participant_trials['image2'].str.split('[',1).str[1].str.strip()
participant_trials['startOri2'] = participant_trials['startOri2'].str.split(']',0).str[0].str.strip()
participant_trials['image1'] = participant_trials['image1'].str.split('[',0).str[0].str.strip()
participant_trials['image2'] = participant_trials['image2'].str.split('[',0).str[0].str.strip()
participant_trials['test1'] = participant_trials['test1'].str.split('[',0).str[0].str.strip()
participant_trials['test2'] = participant_trials['test2'].str.split('[',0).str[0].str.strip()

# converting string (numbers) to datatype float
participant_trials['startOri1'] = participant_trials['startOri1'].astype(float)
participant_trials['startOri2'] = participant_trials['startOri2'].astype(float)


# check that the presentation orientation of the two stimuli is not the same
participant_trials['orientation_match'] = np.where(participant_trials['startOri1'] == participant_trials['startOri2'], 'True', 'False')
#new_participant_trials['orientation_match'] = np.where(new_participant_trials['startOri1'] == new_participant_trials['startOri2'], 'True', 'False')
found = participant_trials[participant_trials['orientation_match'].str.contains('True')]
#found = new_participant_trials[new_participant_trials['orientation_match'].str.contains('True')]
print(found.count())


60
60
66
image1               0
image2               0
test1                0
test2                0
baselinePresent      0
baselinePresent2     0
adjust2Present       0
trial_type           0
swap                 0
startOri1            0
startOri2            0
orientation_match    0
dtype: int64


In [682]:
selected_images

['stimuli/hat04a.png[348.75]',
 'stimuli/stackingring01b.png[326.25]',
 'stimuli/corkscrew03a.png[33.75]',
 'stimuli/barbecue02.png[146.25]',
 'stimuli/pear01.png[11.25]',
 'stimuli/gecko.png[258.75]',
 'stimuli/stapler03b.png[56.25]',
 'stimuli/razor04a.png[146.25]',
 'stimuli/pintofbeer.png[101.25]',
 'stimuli/outdoorheater.png[33.75]']

In [509]:
def set_start_ori(row):
    """
    Function to generate values for the startOri3 and startOri4 columns based on values in the startOri1 and startOri2 columns,
    for rows where swap is No and trial_type is not baseline.
    """
    if row["swap"] == "No":
        if random.random() < 0.5:
            subtract_val = random.randint(1, 180)
            row["startOri3"] = str(float(row["startOri1"]) - subtract_val)
            row['CorrAns'] = 'left'
            if row["startOri2"] != "_":
                if random.random() < 0.5:
                    subtract_val2 = random.randint(1, 180)
                    row["startOri4"] = str(float(row["startOri2"]) - subtract_val2)
                    row['CorrAns2'] = 'left'
                else:
                    add_val1 = random.randint(1, 180)
                    row["startOri4"] = str(float(row["startOri2"]) + add_val1)
                    row['CorrAns2'] = 'right'
            else:
                row["startOri4"] = "_"
                row['CorrAns2'] = "_"
        else:
            add_val2 = random.randint(1, 180)
            row["startOri3"] = str(float(row["startOri1"]) + add_val2)
            row['CorrAns'] = 'right'
            if row["startOri2"] != "_":
                if random.random() < 0.5:
                    subtract_val3 = random.randint(1, 180)
                    row["startOri4"] = str(float(row["startOri2"]) - subtract_val3)
                    row['CorrAns2'] = 'left'
                else:
                    add_va3 = random.randint(1, 180)
                    row["startOri4"] = str(float(row["startOri2"]) + add_va3)
                    row['CorrAns2'] = 'right'
            else:
                row["startOri4"] = "_"
                row['CorrAns2'] = "_"
    elif row["swap"] == "Yes" and row["trial_type"] == "baseline":
        if random.random() < 0.5:
            subtract_val4 = random.randint(1, 180)
            row["startOri3"] = str(float(row["startOri1"]) - subtract_val4)
            row['CorrAns'] = 'left'
        else:
            add_va4 = random.randint(1, 180)
            row["startOri3"] = str(float(row["startOri1"]) + add_va4)
            row['CorrAns'] = 'right'
        row["startOri4"] = "_"
        row['CorrAns2'] = "_"
    else:
        if random.random() < 0.5:
            subtract_val5 = random.randint(1, 180)
            if row["startOri2"] != "_":
                row["startOri3"] = str(float(row["startOri2"]) - subtract_val5)
                row['CorrAns'] = 'left'
                if random.random() < 0.5:
                    subtract_val6 = random.randint(1, 180)
                    row["startOri4"] = str(float(row["startOri1"]) - subtract_val6)
                    row['CorrAns2'] = 'left'
                else:
                    add_va5 = random.randint(1, 180)
                    row["startOri4"] = str(float(row["startOri1"]) + add_va5)
                    row['CorrAns2'] = 'right'
            else:
                row["startOri3"] = "_"
                row['CorrAns'] =  "_"
                row["startOri4"] = "_"
                row['CorrAns2'] = "_"
        else:
            if row["startOri2"] != "_":
                add_va6 = random.randint(1, 180)
                row["startOri3"] = str(float(row["startOri2"]) + add_va6)
                row['CorrAns'] = 'right'
                if random.random() < 0.5:
                    subtract_val7 = random.randint(1, 180)
                    row["startOri4"] = str(float(row["startOri1"]) - subtract_val7)
                    row['CorrAns2'] = 'left'
                else:
                    add_va7 = random.randint(1, 180)
                    row["startOri4"] = str(float(row["startOri1"]) + add_va7)
                    row['CorrAns2'] = 'right'
            else:
                row["startOri3"] = "_"
                row['CorrAns'] =  "_"
                row["startOri4"] = "_"
                row['CorrAns2'] = "_"
    
    return row


In [683]:
# return the new values for startOri3 and startOri4
participant_trials = participant_trials.apply(set_start_ori, axis=1)
participant_trials

,image1,image2,test1,test2,baselinePresent,baselinePresent2,adjust2Present,trial_type,swap,startOri1,startOri2,orientation_match,startOri3,CorrAns,startOri4,CorrAns2
0,stimuli/megaphone.png,stimuli/flipflop01a.png,stimuli/flipflop01a.png,stimuli/megaphone.png,0,1,1,t2,Yes,101.25,78.75,False,201.75,right,10.25,left
1,stimuli/buoy.png,stimuli/cornet.png,stimuli/cornet.png,_,0,1,0,t3,Yes,78.75,326.25,False,410.25,right,55.75,left
2,stimuli/africanelephant.png,stimuli/nail.png,stimuli/nail.png,_,0,1,0,t3,Yes,101.25,236.25,False,208.25,left,234.25,right
3,stimuli/gorilla.png,_,stimuli/gorilla.png,_,1,0,0,baseline,No,281.25,NaN,False,226.25,left,nan,right
4,stimuli/candelabra.png,_,stimuli/candelabra.png,_,1,0,0,baseline,Yes,281.25,NaN,False,348.25,right,_,_
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,stimuli/pylon01.png,_,stimuli/pylon01.png,_,1,0,0,baseline,Yes,213.75,NaN,False,314.75,right,_,_
62,stimuli/witchhat.png,_,stimuli/witchhat.png,_,1,0,0,baseline,Yes,281.25,NaN,False,328.25,right,_,_
63,stimuli/shoppingcart.png,stimuli/redfox.png,stimuli/redfox.png,stimuli/shoppingcart.png,0,1,1,t2,Yes,11.25,33.75,False,201.75,right,-73.75,left
64,stimuli/outdoorheater.png,stimuli/fireextinguisher02.png,stimuli/fireextinguisher02.png,_,0,1,0,t3,Yes,33.75,191.25,False,110.25,left,182.75,right


In [684]:
# assigning the correct arrow direction the partiipant has to click depending on the presentation orientation and the orientation that startOri3/4 (the orientations randomly assigned to test 1& 2).
#assign_direction(participant_trials)

# cleaning up dataframe regarding missing values etc. that psychopy usually does not tolerate in condition files
values = "_"
participant_trials['startOri2'] = participant_trials['startOri2'].fillna(value = values)
participant_trials['startOri4'] = participant_trials['startOri4'].fillna(value = values)
participant_trials['startOri4'] = participant_trials['startOri4'].replace("nan", values)
participant_trials

,image1,image2,test1,test2,baselinePresent,baselinePresent2,adjust2Present,trial_type,swap,startOri1,startOri2,orientation_match,startOri3,CorrAns,startOri4,CorrAns2
0,stimuli/megaphone.png,stimuli/flipflop01a.png,stimuli/flipflop01a.png,stimuli/megaphone.png,0,1,1,t2,Yes,101.25,78.75,False,201.75,right,10.25,left
1,stimuli/buoy.png,stimuli/cornet.png,stimuli/cornet.png,_,0,1,0,t3,Yes,78.75,326.25,False,410.25,right,55.75,left
2,stimuli/africanelephant.png,stimuli/nail.png,stimuli/nail.png,_,0,1,0,t3,Yes,101.25,236.25,False,208.25,left,234.25,right
3,stimuli/gorilla.png,_,stimuli/gorilla.png,_,1,0,0,baseline,No,281.25,_,False,226.25,left,_,right
4,stimuli/candelabra.png,_,stimuli/candelabra.png,_,1,0,0,baseline,Yes,281.25,_,False,348.25,right,_,_
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,stimuli/pylon01.png,_,stimuli/pylon01.png,_,1,0,0,baseline,Yes,213.75,_,False,314.75,right,_,_
62,stimuli/witchhat.png,_,stimuli/witchhat.png,_,1,0,0,baseline,Yes,281.25,_,False,328.25,right,_,_
63,stimuli/shoppingcart.png,stimuli/redfox.png,stimuli/redfox.png,stimuli/shoppingcart.png,0,1,1,t2,Yes,11.25,33.75,False,201.75,right,-73.75,left
64,stimuli/outdoorheater.png,stimuli/fireextinguisher02.png,stimuli/fireextinguisher02.png,_,0,1,0,t3,Yes,33.75,191.25,False,110.25,left,182.75,right


In [685]:
# Identify the duplicate rows
#duplicates = conditions_ltm_[conditions_ltm_.duplicated()]


duplicates = conditions_ltm_[conditions_ltm_.duplicated(subset=['ltm_image'])]
conditions_ltm_ = conditions_ltm_.drop_duplicates()                   

In [686]:
conditions_ltm_


,trial_type,image1,startOri_ltm,ltm_image
0,t3,stimuli/monument.png[258.75],258.75,stimuli/monument.png
1,t3,stimuli/armadillo.png[33.75],33.75,stimuli/armadillo.png
2,baseline,stimuli/maraca02a.png[168.75],168.75,stimuli/maraca02a.png
3,t2,stimuli/flamingo.png[348.75],348.75,stimuli/flamingo.png
4,baseline,stimuli/soapdispenser03b.png[213.75],213.75,stimuli/soapdispenser03b.png
...,...,...,...,...
93,t2,stimuli/chisel01b.png[348.75],348.75,stimuli/chisel01b.png
94,t2,stimuli/badmintonracket.png[168.75],168.75,stimuli/badmintonracket.png
95,t2,stimuli/comb02b.png[326.25],326.25,stimuli/comb02b.png
97,t2,stimuli/shoppingcart.png[11.25],11.25,stimuli/shoppingcart.png


In [687]:
#len(participant_trials)
# export my excel files for the working memory and ltm memory task
file_name_WM = 'conditions_428.xlsx'
participant_trials.to_excel(file_name_WM, index=False)

file_name_LTM = 'conditions_ltm_428.xlsx'
conditions_ltm_.to_excel(file_name_LTM, index=False)

In [688]:
duplicates

,trial_type,image1,startOri_ltm,ltm_image
80,t3,stimuli/hat04a.png[348.75],348.75,stimuli/hat04a.png
84,t3,stimuli/stackingring01b.png[326.25],326.25,stimuli/stackingring01b.png
85,t3,stimuli/corkscrew03a.png[33.75],33.75,stimuli/corkscrew03a.png
86,t3,stimuli/barbecue02.png[146.25],146.25,stimuli/barbecue02.png
87,t3,stimuli/pear01.png[11.25],11.25,stimuli/pear01.png
88,t3,stimuli/gecko.png[258.75],258.75,stimuli/gecko.png
89,t3,stimuli/stapler03b.png[56.25],56.25,stimuli/stapler03b.png
90,t3,stimuli/razor04a.png[146.25],146.25,stimuli/razor04a.png
96,t3,stimuli/pintofbeer.png[101.25],101.25,stimuli/pintofbeer.png
98,t3,stimuli/outdoorheater.png[33.75],33.75,stimuli/outdoorheater.png


In [ ]:
## Older function down here, just to keep


In [268]:
def assign_direction(df):
    # Replace '_' with NaN in startOri2 column
    df['startOri2'] = df['startOri2'].replace('_', np.nan)
    
    # Convert columns to numeric data type
    df['startOri1'] = pd.to_numeric(df['startOri1'], errors='coerce')
    df['startOri2'] = pd.to_numeric(df['startOri2'], errors='coerce')
    df['startOri3'] = pd.to_numeric(df['startOri3'], errors='coerce')
    df['startOri4'] = pd.to_numeric(df['startOri4'], errors='coerce')

    # Assign directions based on comparisons and 'swap' condition
    df['direction1'] = df.apply(lambda row: 'right' if (row['startOri3'] < row['startOri1'] and
                                                        row['swap'] == 'No' and
                                                        (row['trial_type'] != 'baseline' or pd.notna(row['startOri2']))) else
                                              'right' if (row['startOri3'] < row['startOri2'] and
                                                           row['swap'] == 'Yes' and
                                                           (row['trial_type'] != 'baseline' or pd.notna(row['startOri2']))) else
                                              'left',
                                axis=1)
    
    def compare_direction(row):
        if row['swap'] == 'No':
            if row['trial_type'] == 'baseline' and pd.isna(row['startOri2']):
                return '_'
            else:
                return 'right' if (row['startOri4'] < row['startOri2'] and
                                  (row['trial_type'] != 'baseline' or pd.notna(row['startOri2']))) else 'left'
        elif row['swap'] == 'Yes':
            return 'right' if (row['startOri4'] < row['startOri1'] and
                              (row['trial_type'] != 'baseline' or pd.notna(row['startOri2']))) else 'left'
        else:
            return '_'
    
    df['direction2'] = df.apply(compare_direction, axis=1)
    
    return df

In [ ]:
participant_trials['startOri3'] = [random.randint(0, 360) for _ in range(participant_trials.shape[0])]
participant_trials['startOri4'] = [random.randint(0, 360) for _ in range(participant_trials.shape[0])]
participant_trials